In [ ]:
import sympy as sp 
import numpy as np
import matplotlib.pyplot as plt
import cayley_schreier as cs

In [ ]:
#define symbols
t1, t2, m = sp.symbols('t1 t2 m', real=True) #lattice parameters
k1, k2 = sp.symbols('k1 k2', real=True) #momentum components
syms = (k1, k2, t1, t2, m) #all symbols

In [ ]:
#check C6 symmetry in the x-y basis
basis = "xy" #choose between "12" and "xy"
Hk = cs.create_honeycomb_hamiltonian(syms, basis) 

cs.check_periodicity(Hk, syms) #check periodicity (shouldn't be periodic in k1 and k2)

#define C6 operator
s0, sx, sy, sz = cs.Pauli()
V = s0/2 - sp.I/2*(sx+sy+sz) 
C6_unitary = cs.TP(sx, V)
diff = cs.check_C6(Hk, C6_unitary, [k1,k2])
diff

In [ ]:
#Compute the zigzag spectrum in the a1-a2 basis
basis = "12" 
Hk = cs.create_honeycomb_hamiltonian(syms, basis) 
cs.check_periodicity(Hk, syms) #check periodicity (shouldn't be periodic in k1 and k2)

In [ ]:
#need to make the Hamiltonian periodic in k1 for the surface calculation
#define gauge transformation that makes the Hamiltonian periodic in k1
Dk = sp.zeros(4,4)
Dk[2:, 2:] = sp.eye(2) * sp.exp(sp.I * k1 * sp.Rational(1,3)) * sp.exp(-sp.I * k2 * sp.Rational(1,3))
Dk[:2, :2] = sp.eye(2)

#gauge transformation
Hk_tilde = Dk @ Hk @ Dk.H

#check periodicity again
cs.check_periodicity(Hk_tilde, syms) #check periodicity (should be periodic in k1 but not in k2)

In [ ]:
#define parameters for surface Hamiltonian calculation
params = {
    "N1": 32,
    "N2": 128,
    "num_bands": 4,
    "num_occupied": 2,
    "t1": 1.0,
    "t2": 1.0,
    "m": 1.0
}
#surfaceBZ 
K2s = np.linspace(0, 2*np.pi, params["N2"], endpoint=False)

#compute the surface Hamiltonian 
H_obc = cs.get_surface_hamiltonian(Hk_tilde, k1, params["N1"], params["num_bands"])

#evaluate the surface Hamiltonian at t1=t2=m=1
H_obc_subs = H_obc.subs({t1:params["t1"], t2:params["t2"], m:params["m"]})
H_obc_func = sp.lambdify(k2, H_obc_subs, 'numpy')

In [ ]:
eigenvals, eigenvecs = cs.spectrum(H_obc_func, [K2s], params["num_bands"]*params["N1"])
eigenvecs, eigenvals = cs.continuous_bands_1d(vals = eigenvals, vecs = eigenvecs)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
ax.plot(K2s, eigenvals, color='black', alpha = 0.3)
ax.set_xlabel(r'$k_{||}$', fontsize = 14, labelpad = -0)
ax.set_ylabel(r'$\epsilon$', fontsize = 14, labelpad = -5)
ax.set_xticks([0, np.pi, 2*np.pi])
ax.set_xticklabels([r"$\mathsf{0}$", r"$\mathsf{\pi}$", r"$\mathsf{2\pi}$"], fontsize=14)
ax.set_yticks([-3, 0, 3])
ax.set_yticklabels([r"$\mathsf{-3}$", r"$\mathsf{0}$", r"$\mathsf{3}$"], fontsize=14)
#plt.savefig("../figures/kanemele_edge_spectrum.svg", dpi=1000, bbox_inches='tight')
plt.show()

In [ ]:
params = {
    "N1": 121,
    "N2": 121,
    "num_bands": 4,
    "num_occupied": 2,
    "t1": 1.0,
    "t2": 1.0,
    "m": 1.0
}

In [ ]:
#Compute the zigzag spectrum in the a1-a2 basis
basis = "12" 
Hk = cs.create_honeycomb_hamiltonian(syms, basis) 

Dk = sp.zeros(4,4)
Dk[2:, 2:] = sp.eye(2) * sp.exp(sp.I * k1 * sp.Rational(1,3)) * sp.exp(-sp.I * k2 * sp.Rational(1,3))
Dk[:2, :2] = sp.eye(2)
Hk_tilde = Dk @ Hk @ Dk.H
cs.check_periodicity(Hk_tilde, syms) #check periodicity (should be periodic in k1 and k2)

In [ ]:
Hk_tilde_subs = Hk_tilde.subs({t1:params["t1"], t2:params["t2"], m:params["m"]})
Hk_tilde_func = sp.lambdify((k1, k2), Hk_tilde_subs, 'numpy')
K1s = np.linspace(0, 2*np.pi, params["N1"], endpoint=False) + 1e-10 #small shift is required for proper separation of bands
K2s = np.linspace(0, 2*np.pi, params["N2"], endpoint=False) + 1e-10
eigenvals, eigenvecs = cs.spectrum(Hk_tilde_func, [K1s, K2s], params["num_bands"])

In [ ]:
phases, wilson_basis = cs.wilson.wilson_loop_eigs(vecs = eigenvecs[...,:2], axis = -3)

In [ ]:
fig, ax = plt.subplots(figsize = (3,3))
uw_0 = np.unwrap(phases[:,0], period = 1)
uw_1 = np.unwrap(phases[:,1], period = 1)
shift_0 = 1 if np.mean(uw_0) < -0.1 else 0
shift_1 = 1 if np.mean(uw_1) < -0.1 else 0
plt.plot(K2s, uw_0 + shift_0, c = "k")
plt.plot(K2s, uw_1 + shift_1, c = "k")

plt.xlabel(r"$k$", fontsize = 14)
plt.ylabel(r"$\nu$", fontsize = 14)
ax.set_xticks([0, np.pi, 2*np.pi])
ax.set_xticklabels([r"$\mathsf{0}$", r"$\mathsf{\pi}$", r"$\mathsf{2\pi}$"], fontsize=14)
ax.set_yticks([0, 0.5, 1])
ax.set_yticklabels([r"$0$", r"$0.5$", r"$1$"], fontsize = 14)

plt.show()